In [1]:
import torch # tested with torch v1.13, python v1.9
from torch import nn
from torch.nn import functional as F
from torch import distributions as dist
import numpy as np

Importing classes and functions for l-ACNF. See `l_acnf.py` script for their implementations

In [2]:
from l_acnf import Lattice, l_ACNF, sample_fn, logprob_fn, train

Initialize lattice, device and model

In [3]:
lattice = Lattice(16, 5.0, True)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
net = l_ACNF(8, 128).to(device)

Train model for given epochs, batchsize and learning rate:

In [ ]:
epochs = 9000
batch = 128
train(lattice, net, batch, epochs, 2e-3, 1500, device)

Or uncomment below cell and load pre-trained weights:

In [5]:
net.load_state_dict(torch.load("lacnf_d_8.pth"))

<All keys matched successfully>

Import helper functions for Metropolis-Hastings sampling using the trained model. See `mc_sample.py` for implementation details

In [7]:
from mc_sample import metropolis, greens_function, pole_mass, susceptibilty, ising_energy

Initialize observables

In [ ]:
m_p_mean = torch.zeros([7])
m_p_std = torch.ones([7])

chi_2_mean = torch.zeros([7])
chi_2_std = torch.ones([7])

E_mean = torch.zeros([7])
E_std = torch.ones([7])

rej = []
chi_1 = []
chi_2 = []
E = []

Evaluate observables at every Monte Carlo step as well as their means and standard errors via moving block bootstrap

In [ ]:
g = [6.008, 5.55, 5.276, 5.113, 4.99, 4.89, 4.82]
T = 100_000

for (j,L) in enumerate(range(8, 21, 2)):
    lattice = Lattice(L, g[j], True)
    phi, r, c1, c2, e = metropolis(lattice, net, 2500, T, device)
    
    rej.append(r)
    chi_1.append(c1)
    chi_2.append(c2)
    E.append(e)
    
    # Moving block bootstrap
    bin = 100
    boxes = [torch.arange(i, i+bin, device=device) 
            for i in range(T-bin+1)]
    
    m_p = []
    chi_22 = torch.zeros([bin])
    E2 = torch.zeros([bin])
    
    for n in range(bin):
        r = np.random.randint(0, T-bin+1, (T//bin,))
        indx = torch.cat([boxes[n] for n in r], dim=0)
        
        phi2 = phi[indx, :, :]
        G = greens_function(phi2, device)
        G2 = G.mean(1)
        
        m_p.append(pole_mass(G2))
        chi_22[n] = susceptibilty(G)
        E2[n] = ising_energy(G)
    
    m_p = torch.stack(m_p, dim=0)
    m_p_mean[j] = m_p.mean(0)[1:].mean()*L
    m_p_std[j] = m_p.std(0)[1:].mean()*L
    
    chi_2_mean[j] = chi_22.mean()
    chi_2_std[j] = chi_22.std()
    E_mean[j] = E2.mean()
    E_std[j] = E2.std()

rej = torch.stack(rej, dim=0)
chi_1 = torch.stack(chi_1, dim=0)
chi_2 = torch.stack(chi_2, dim=0)
E = torch.stack(E, dim=0)

In [ ]:
print(m_p_mean, m_p_std) # Average Pole mass
print(chi_2_mean, chi_2_std) # 2 point susceptibility
print(E_mean, E_std) # Ising energy

Evaluate autocorrelation functions and times wrt acceptance statistics and various observables

In [ ]:
t_max = 100
rho_acc = torch.zeros([rej.shape[0], t_max])
for t in range(1, t_max+1):
    pool_op = torch.nn.MaxPool1d(t, 1)
    rho_acc[:, t-1] = (1-pool_op((1-rej).unsqueeze(1))).mean([1,2])
t_int_acc = 0.5 + rho_acc.sum(1)

In [ ]:
rho_chi_1 = torch.zeros([chi_1.shape[0], t_max])
rho_chi_2 = torch.zeros([chi_2.shape[0], t_max])
rho_E = torch.zeros([E.shape[0], t_max])

for t in range(1, t_max+1):
    chi_1_mean2 = chi_1.mean(1, keepdim=True)
    covar = (chi_1[:, :-t]-chi_1_mean2)*(chi_1[:, t:]-chi_1_mean2)
    rho_chi_1[:, t-1] = covar.mean(1)/chi_1.var(1)
    
    chi_2_mean2 = chi_2.mean(1, keepdim=True)
    covar = (chi_2[:, :-t]-chi_2_mean2)*(chi_2[:, t:]-chi_2_mean2)
    rho_chi_2[:, t-1] = covar.mean(1)/chi_2.var(1)
    
    E_mean2 = E.mean(1, keepdim=True)
    covar = (E[:, :-t]-E_mean2)*(E[:, t:]-E_mean2)
    rho_E[:, t-1] = covar.mean(1)/E.var(1)

t_int_chi_1 = 0.5 + rho_chi_1.sum(1)
t_int_chi_2 = 0.5 + rho_chi_2.sum(1)
t_int_E = 0.5 + rho_E.sum(1)

In [ ]:
print(rho_acc)
print(t_int_acc) # Acceptance statistics

print(rho_chi_1)
print(t_int_chi_1) # G(0) or 1-point susceptibility

print(rho_chi_1)
print(t_int_chi_1) # 2-point susceptibility

print(rho_E)
print(t_int_E) # Ising energy